In [ ]:
from cvxopt import matrix, solvers
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import time

SEED = 462
np.random.seed(SEED)

data_path = os.path.join("..", "data_workflow_notebooks", "data", "tabular")

In [ ]:
class Dataset:
    """
    This class is taken directly from our previous submission.
    """
    def __init__(self, train_path=None, test_path=None):
        self.train_path = train_path
        self.test_path = test_path
    
    def load_csv(self, path):
        data = pd.read_csv(path).to_numpy()
        X, Y_str = data[:, :-1], data[:, -1]  # separate data and target
        n_examples = len(Y_str)
        Y = np.zeros(n_examples)
        for i in range(n_examples):
            category = Y_str[i]
            if category == "banana":
                Y[i] = 0
            elif category == "carrot":
                Y[i] = 1
            elif category == "cucumber":
                Y[i] = 2
            elif category == "mandarin":
                Y[i] = 3
            else:
                Y[i] = 4
        
        return X.astype(float), Y.astype(float)
    
    def get_data(self):
        X_train, Y_train = self.load_csv(self.train_path)
        X_test, Y_test   = self.load_csv(self.test_path)
        
        return (X_train, Y_train), (X_test, Y_test)